# Derivatives and ECG Signals

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import numpy.random as ra
import matplotlib.pyplot as plt
import pandas as pd
import os

from ipywidgets import interact, interactive, fixed
from IPython.display import clear_output
import ipywidgets as widgets

DATADIR = os.path.join(os.path.expanduser("~"),
                    "work",
                    "Physionet", "MITDB")
os.path.exists(DATADIR)

![ECG annotation](https://goo.gl/l4Rlq8)

In [ ]:
ecg_files = os.listdir(DATADIR)
type(ecg_files)

In [ ]:
def get_series(f):
    
    data = \
    pd.read_table(f, header=None).apply(
        lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return data
def _view_series(root="", f="", num_points=200):
    data = get_series(os.path.join(root,f))
    plt.plot(data[1][:num_points])
    plt.show()
interactive_plot = interactive(_view_series, root=fixed(DATADIR), f=ecg_files, num_points=(200, 1000, 50))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


In [ ]:
data = get_series(os.path.join(DATADIR, "100.txt"))
data.head()

## Using [Matplotlib](http://matplotlib.org/) to plot simple time-series data

The most common plotting package in Python is Matplotlib. The simplest plot in matplotlib is a line plot via the [``plot``](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot) function.

### Select a subset of the signal

In [ ]:
t = data[0][:300]
s = data[1][:300]
plt.plot(t, s )

### Formatting the line style

I can specify the line style and color with a format string. For example, I can specify the color.

In [ ]:
plt.plot(t, s, "r")

We can specify a number of keyword arguments to control the plot in more detail.

In [ ]:
plt.plot(t[::2], s[::2], linestyle='dashdot', marker='o', color="r", markersize=10, label="ECG")
plt.legend()

## Controlling figure layout with [subplots](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.subplots)

In [ ]:
f, ax = plt.subplots(3,1)
f.set_size_inches(8,12)
t = data[0][:300]
s = data[1][:300]
ax[0].plot(t, data[1][:300], 'b')
ax[1].plot(t, np.gradient(s), 'r')
ax[2].plot(t, np.gradient(np.gradient(s)), 'g')

## The Data are Noisy 
#### Computing derivatives tends to be sensitive to noise
#### Try smoothing first

The SciPy package has a number of functions for smoothing signals and images. The [signal](http://docs.scipy.org/doc/scipy/reference/signal.html) module includes common one-dimensional filtering functions. The [ndimage.filters](scipy.ndimage.filters) module includes a number of functions for smoothing N-dimensional signals; the name is derived from 2D signals (images).

In [ ]:
import scipy.signal as signal
import scipy.ndimage.filters as filters

#### We will explore two smooth functions

* [Median smoothing](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.medfilt.html#scipy.signal.medfilt)
* [Gaussian smoothing](http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.gaussian_filter.html)

### Median Filtering

#### What is the median of 

* 1, 5, 3, 7, 2?

Median filtering is a kind of non-linear filtering that is good at preserving edges and other discontinuities in the data. The [medfilter]() function takes a positional argument that is the signal that you want to filter (smooth) and a keyword argument ``kernel_size`` (default 3) that defines how many points you want to use to calculate the median.

![Median Filter algorithm](https://upload.wikimedia.org/wikipedia/commons/f/f4/Median_filter_pseudocode.png)

![Example median filter](https://upload.wikimedia.org/wikipedia/commons/1/1d/Medianfilterp.png)

### Play with changing `kernel_width`

In [ ]:
kernel_width = 9
f, ax = plt.subplots(3,1)
f.set_size_inches(8,12)
t = data[0][:300]
s = signal.medfilt(data[1][:300], kernel_size=kernel_width)
    
ax[0].plot(t, s, 'b')
ax[1].plot(t, np.gradient(s), 'r')
ax[2].plot(t, np.gradient(np.gradient(s)), 'g')

### Gaussian Smoothing

The [``gaussian_filter``](). Smoothing with the Gaussian function is described as a convolution of the original signal with a Gaussian function.

### What is convolution

![Visualization of convolution](https://goo.gl/FWGliL)

Convolutions is a function that takes as input two signals ($f$) and ($g$) and returns as output a new signal that is obtained by  
1. flipping $f$
1. shifting $g1$
2. multiplying the shifted and flipped $f$ by $g$.
1. summing up the value sof the multiplication.

This can be written mathematically as

$$
(f * g)(t) \overset{\Delta}{=} \int_{-\infty}^\infty f(t-\tau)g(\tau)d\tau
$$

### What is a [Gaussian](https://en.wikipedia.org/wiki/Gaussian_function)

The Gaussian is a function that you will encounter over and over.

SciPy has a [Gaussian window function](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.gaussian.html)

In [ ]:
plt.plot(signal.gaussian(100,10))

### Why might this be a good function for smoothing?

In [ ]:
gaussian_width = 3
f, ax = plt.subplots(3,1)
f.set_size_inches(8,12)
t = data[0][:300]
s = filters.gaussian_filter(data[1][:300], gaussian_width)
    
ax[0].plot(t, s, 'b')
ax[1].plot(t, np.gradient(s), 'r')
ax[2].plot(t, np.gradient(np.gradient(s)), 'g')

In [ ]:
gaussian_width = 3
f, ax = plt.subplots(3,1)
f.set_size_inches(8,12)
t = data[0][:300]
s = data[1][:300]
    
ax[0].plot(t, filters.gaussian_filter(s, gaussian_width, order=0), 'b')
ax[1].plot(t, filters.gaussian_filter(s, gaussian_width, order=1), 'r')
ax[2].plot(t, filters.gaussian_filter(s, gaussian_width, order=2), 'g')

## Exercise: How could you use 1st and 2nd derivatives to detect the *R* wave of the ECG?

#### Hints:

* Use the [``np.where``](http://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html) function in conjunction with the [``np.logical_and``](http://docs.scipy.org/doc/numpy/reference/generated/numpy.logical_and.html#numpy.logical_and) function.

In [ ]:
plt.plot(t, s*(filters.gaussian_filter(s, gaussian_width, order=2) < 0))

In [ ]:
gaussian_width = 7
mask = np.where(np.logical_and(filters.gaussian_filter(s, gaussian_width, order=1) > 0,
                               filters.gaussian_filter(s, gaussian_width, order=2) < 0),
                1, 0)

plt.plot(t, s*mask)

### Make an Interactive Exploration of the Derivatives